Read the case, department, and source data into their own spark dataframes.

Let's see how writing to the local disk works in spark:

Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json

Inspect your folder structure. What do you notice?

Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

In [2]:
source = spark.read.csv("source_copy.csv", sep=",", header=True, inferSchema=True)

In [3]:
department = spark.read.csv("dept_copy.csv", sep=",", header=True, inferSchema=True)

In [4]:
case = spark.read.csv("case_copy.csv", sep=",", header=True, inferSchema=True)

In [5]:
source.show(5)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
|   106463| Richard Sanchez|
|   119403|  Betty De Hoyos|
|   119555|  Socorro Quiara|
+---------+----------------+
only showing top 5 rows



In [6]:
df_source_jason = source.write.json("sources_json", mode="overwrite")

In [7]:
df_source_csv = source.write.csv("sources_csv", mode="overwrite")

In [8]:
source.dtypes

[('source_id', 'string'), ('source_username', 'string')]

In [9]:
department.dtypes

[('dept_division', 'string'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'string')]

In [10]:
case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

In [11]:
case = case.withColumn("case_closed", expr('case_closed == "YES"')).withColumn(
    "case_late", expr('case_late == "YES"')
)

In [12]:
case = case.withColumnRenamed("SLA_due_date", "case_due_date")

In [13]:
fmt = "M/d/yy H:mm"

case = (
    case.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_closed_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_due_date", fmt))
)

In [14]:
case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('case_due_date', 'timestamp'),
 ('case_late', 'boolean'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

How old is the latest (in terms of days past SLA) currently open issue? 

How long has the oldest (in terms of days since opened) currently opened issue been open?

How many Stray Animal cases are there?

How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

Convert the council_district column to a string column.

Extract the year from the case_closed_date column.
Convert num_days_late from days to hours in new columns num_hours_late.

Join the case data with the source and department data.

Are there any cases that do not have a request source?

What are the top 10 service request types in terms of number of requests?

What are the top 10 service request types in terms of average days late?
Does number of days late depend on department?
How do number of days late depend on department and request type?

In [15]:
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean

In [16]:
case.filter(case.case_closed== False).select(max(case.SLA_days)).show()

+-------------+
|max(SLA_days)|
+-------------+
|   1419.00191|
+-------------+



In [33]:
case.select(
            datediff(
                      case.filter(case.case_closed==False).select(min(case.case_opened_date)),
                      case.select(max(case.case_closed_date))
                    )
          ).show()

AttributeError: 'DataFrame' object has no attribute 'astype'

In [32]:
#How long has the oldest (in terms of days since opened) currently opened issue been open?
start = case.filter(case.case_closed==False).select(min(case.case_opened_date).alias('date1')).toPandas()
end = case.select(max(case.case_closed_date).alias('date2')).toPandas()


print(start.date1)
print(end.date2)

datediff(start.date1,end.date2)

0   2017-01-01 13:48:00
Name: date1, dtype: datetime64[ns]
0   2018-08-08 10:38:00
Name: date2, dtype: datetime64[ns]


TypeError: Invalid argument, not a string or column: 0   2017-01-01 13:48:00
Name: date1, dtype: datetime64[ns] of type <class 'pandas.core.series.Series'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.

In [ ]:
# How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?
case.filter(case.dept_division=='Field Operations').filter(case.service_request_type!="Officer Standby").count()

In [ ]:
# How many Stray Animal cases are there?
case.select(case.service_request_type=='Stray Animal').count()

Convert the council_district column to a string column.

In [ ]:
case = case.withColumn('council_distrtict',case.council_district.cast("string"))

Extract the year from the case_closed_date column. 

Convert num_days_late from days to hours in new columns num_hours_late.

In [ ]:
case.select(case.case_closed_date[1:4].alias('year')).show(5)

In [ ]:
case.withColumn('num_days_late', expr('num_days_late * 24 AS num_hours_late')).show()

In [ ]:
case.select("num_hours_late").show(5)